In [23]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

# Load the transaction dataset
data = pd.read_csv('mock_transactions.csv')

# Preprocessing the dataset
# Selecting relevant columns for RNN training
features = ['Amount', 'Latitude', 'Longitude', 'Time']
target = 'Is Fraud?'

# Extract time-based features (hour, day of week, etc.)
data['Time'] = pd.to_datetime(data['Time'])
data['Hour'] = data['Time'].dt.hour
data['DayOfWeek'] = data['Time'].dt.dayofweek
data['Day'] = data['Time'].dt.day

# Update features list to include new time-based features
features = ['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']

# Scaling numerical features
scaler = MinMaxScaler()
data[['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']] = scaler.fit_transform(
    data[['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']]
)

# Create feature and target arrays
X = data[features].values
y = data[target].values

# Reshape target for compatibility
y = np.reshape(y, (-1, 1))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=78)

# Reshape the input data for LSTM (samples, timesteps, features)
# Assuming each transaction is independent (timesteps=1)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Building the RNN model
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.9))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.9))

model.add(Dense(units=256, activation='relu'))  # Hidden dense layer
model.add(Dropout(0.9))
model.add(Dense(units=1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Add EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5)

# Print classification report and AUC
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, y_pred)}")

# Save the model
model.save('fraud_detection_rnn.h5')

C:\Users\Thoma\AppData\Local\Temp\ipykernel_27492\3732165561.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time'] = pd.to_datetime(data['Time'])


Epoch 1/50


c:\Users\Thoma\anaconda3\envs\dev\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3833/3833 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.7928 - loss: 0.5184 - val_accuracy: 0.8164 - val_loss: 0.4666
Epoch 2/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8112 - loss: 0.4843 - val_accuracy: 0.8152 - val_loss: 0.4666
Epoch 3/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.8119 - loss: 0.4785 - val_accuracy: 0.8170 - val_loss: 0.4663
Epoch 4/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8131 - loss: 0.4778 - val_accuracy: 0.8213 - val_loss: 0.4641
Epoch 5/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.8136 - loss: 0.4767 - val_accuracy: 0.8222 - val_loss: 0.4634
Epoch 6/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8145 - loss: 0.4746 - val_accuracy: 0.8201 - val_loss: 0.4627
Epoch 7/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8131 - loss: 0.4756 - val_accuracy: 0.8232 - val_loss: 0.4615
Epoch 8/50
3833/3833 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8174 - loss: 0.4706 - val

              precision    recall  f1-score   support

           0       0.83      0.97      0.90     23849
           1       0.78      0.32      0.46      6814

    accuracy                           0.83     30663
   macro avg       0.80      0.65      0.68     30663
weighted avg       0.82      0.83      0.80     30663

AUC: 0.6485282401777853


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load the dataset
data = pd.read_csv('mock_transactions.csv')

# Drop sensitive features
data = data.drop(['Card Number', 'CVV', 'Expires'], axis=1)

# Encode categorical features
label_encoders = {}
for col in ['Errors?', 'Has Chip', 'City', 'State']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Extract time-based features from 'Time'
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S', errors='coerce')
data['Hour'] = data['Time'].dt.hour

# Handle mixed date formats for 'Date'
data['Date'] = pd.to_datetime(data['Date'], format='mixed', errors='coerce')
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Day'] = data['Date'].dt.day

# Update feature list
# Update feature list to exclude certain columns
features = ['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']  # Removed specified columns

# Separate features and target
X = data[features]
y = data[target]

# Normalize numerical features
scaler = StandardScaler()
X.loc[:, ['Amount', 'Latitude', 'Longitude']] = scaler.fit_transform(X[['Amount', 'Latitude', 'Longitude']])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=17)

# Define hyperparameters grid
param_grid = {
    'n_estimators': [500, 100],  # Fewer options
    'max_depth': [50, 60],       # Limit depth options
    'min_samples_split': [5],    # Use a single value
    'min_samples_leaf': [2],     # Use a single value
    'max_features': ['sqrt']     # Use a single option
}

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=17)

# Apply GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-8, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Use best model
best_rf_model = grid_search.best_estimator_

# Train the best model
best_rf_model.fit(X_train, y_train)

# Make predictions
y_pred = best_rf_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, y_pred)}")


model.save('fraud_detection_RF.h5')


Fitting 3 folds for each of 4 candidates, totalling 12 fits


c:\Users\Thoma\anaconda3\envs\dev\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: {'max_depth': 60, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 500}


              precision    recall  f1-score   support

           0       0.84      0.97      0.90     95249
           1       0.77      0.35      0.48     27401

    accuracy                           0.83    122650
   macro avg       0.81      0.66      0.69    122650
weighted avg       0.82      0.83      0.81    122650

AUC: 0.6601550672026535
